# What is Pneumonia?
**Pneumonia is an inflammatory condition of the lung affecting primarily the small air sacs known as alveoli.Symptoms typically include some combination of productive or dry cough, chest pain, fever and difficulty breathing. The severity of the condition is variable. Pneumonia is usually caused by infection with viruses or bacteria and less commonly by other microorganisms, certain medications or conditions such as autoimmune diseases.Risk factors include cystic fibrosis, chronic obstructive pulmonary disease (COPD), asthma, diabetes, heart failure, a history of smoking, a poor ability to cough such as following a stroke and a weak immune system. Diagnosis is often based on symptoms and physical examination. Chest X-ray, blood tests, and culture of the sputum may help confirm the diagnosis.The disease may be classified by where it was acquired, such as community- or hospital-acquired or healthcare-associated pneumonia.**


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os


# Importing the necessary libraries

In [ ]:
import matplotlib.pyplot as plt  #  visualizations like histograms, scatter plots, etc 
import seaborn as sns  #  to visualize random distributions
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau
import cv2  #pip install opencv-python # powerful library for computer vision tasks such as image processing, etc 
import os

# Description of the Pneumonia Dataset
**The dataset is organized into 3 folders (train, test, val) and contains subfolders for each image category (Pneumonia/Normal). There are 5,863 X-Ray images (JPEG) and 2 categories (Pneumonia/Normal).
Chest X-ray images (anterior-posterior) were selected from retrospective cohorts of pediatric patients. All chest X-ray imaging was performed as part of patients’ routine clinical care.
For the analysis of chest x-ray images, all chest radiographs were initially screened for quality control by removing all low quality or unreadable scans. The diagnoses for the images were then graded by two expert physicians before being cleared for training the AI system. In order to account for any grading errors, the evaluation set was also checked by a third expert.**

The function preprocess_data processes a dataset of chest X-ray images, converting them to grayscale, resizing them to 150x150 pixels, and labeling them as either pneumonia or normal based on their directory. The output is a list of image-label pairs, which can then be used for training a machine learning model.

In [ ]:
labels = ['PNEUMONIA', 'NORMAL']
img_size = 150
def preprocess_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                #print(np.shape(img_arr))
                resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
                #print(np.shape(resized_arr))
                data.append([resized_arr, class_num]) 
            except Exception as e:
                print(e)
    return data  

# Loading the Dataset

In [ ]:
train = preprocess_data(r'E:\Dataset_chest_xray/train')
print(train[0])

In [ ]:
test = preprocess_data(r'E:\Dataset_chest_xray/test')
val = preprocess_data(r'E:\Dataset_chest_xray/val')

# Data Visualization & Preprocessing

In [ ]:
# Convert list to a pandas Series
l = []
for i in train:
    if i[1] == 0:
        l.append("Pneumonia")
    else:
        l.append("Normal")

# Create a pandas DataFrame
label_df = pd.DataFrame(l, columns=['Label'])
print(label_df.head())

# Plot using Seaborn
sns.set_style('darkgrid')
sns.countplot(x='Label', data=label_df)
plt.title("Distribution of Labels in the Training Set")
plt.show()

**The data seems imbalanced . To increase the no. of training examples, we will use data augmentation**

**Previewing the images of both the classes**

In [ ]:
plt.figure(figsize = (5,5))
plt.imshow(train[0][0], cmap='gray')
plt.title(labels[train[0][1]])

plt.figure(figsize = (5,5))
plt.imshow(train[-1][0], cmap='gray')
plt.title(labels[train[-1][1]])

In [ ]:
x_train = []
y_train = []

x_val = []
y_val = []

x_test = []
y_test = []

for feature, label in train:
    x_train.append(feature)
    y_train.append(label)

for feature, label in test:
    x_test.append(feature)
    y_test.append(label)
    
for feature, label in val:
    x_val.append(feature)
    y_val.append(label)

**We perform a grayscale normalization to reduce the effect of illumination's differences.Moreover the CNN converges faster on [0..1] data than on [0..255].**

In [ ]:
# Normalize the data
x_train = np.array(x_train) / 255
x_val = np.array(x_val) / 255
x_test = np.array(x_test) / 255

print(x_train[0])
print(np.shape(x_train))

reshape foramat -->reshape(number_of_images/batch_size, height, width, channels); 
-1 --> sets the number of images in x_train while maintaining the overall array size
150,150 --> image size; 
1 --> indicates the number of channels in the image. 
For grayscale images, there is only one channel. 
If the images were in RGB format, this value would be 3 (for red, green, and blue channels).

In [ ]:
# resize data for deep learning (CNN models)

x_train = x_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)
print(np.shape(x_train))
print(x_train[0])

x_val = x_val.reshape(-1, img_size, img_size, 1)
y_val = np.array(y_val)

x_test = x_test.reshape(-1, img_size, img_size, 1)
y_test = np.array(y_test)

# Data Augmentation
**In order to avoid overfitting problem, we need to expand artificially our dataset. We can make your existing dataset even larger. The idea is to alter the training data with small transformations to reproduce the variations.
Approaches that alter the training data in ways that change the array representation while keeping the label the same are known as data augmentation techniques. Some popular augmentations people use are grayscales, horizontal flips, vertical flips, random crops, color jitters, translations, rotations, and much more.
By applying just a couple of these transformations to our training data, we can easily double or triple the number of training examples and create a very robust model.**

In [ ]:
# With data augmentation to prevent overfitting and handling the imbalance in dataset
# no change has been done for this dataset

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1) 


datagen.fit(x_train)

For the data augmentation, i choosed to :
1. Randomly rotate some training images by 30 degrees 
2. Randomly Zoom by 20% some training images
3. Randomly shift images horizontally by 10% of the width 
4. Randomly shift images vertically by 10% of the height 
5. Randomly flip images horizontally.
Once our model is ready, we fit the training dataset.

# Training the Model

In [ ]:
# starts a NN where layers are added one after another.
model = Sequential()

# First Conv2D Layer:
# Adds a 2D convolution layer (Conv2D) with 32 filters, each of size (3,3). 
# It uses ReLU as the activation function, and the input data shape is (150,150,1) (grayscale image).
# BatchNormalization() normalizes the data to help the model learn faster.
# MaxPool2D reduces the size of the data by pooling, which helps focus on important features 
# (reduces data by half with strides of 2).

model.add(Conv2D(32 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (150,150,1)))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

# Second Conv2D Layer:
# Adds a 2D convolution layer with 64 filters and similar parameters.
# Dropout(0.1) drops out 10% of the neurons randomly to prevent overfitting.
# Followed by BatchNormalization() and MaxPool2D() as before.

model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.1))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

# Third Conv2D Layer:
# Adds another Conv2D layer with 64 filters.
# BatchNormalization() and MaxPool2D() for normalization and downsampling.

model.add(Conv2D(64 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

# Fourth Conv2D Layer:
# Adds a Conv2D layer with 128 filters, followed by Dropout(0.2), BatchNormalization(), and MaxPool2D().

model.add(Conv2D(128 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

# Fifth Conv2D Layer:
# Adds a Conv2D layer with 256 filters, followed by Dropout(0.2), BatchNormalization(), and MaxPool2D().

model.add(Conv2D(256 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

# Flattening the Data:
# Flatten() transforms the 2D data into 1D, which is necessary for the next dense layers.
model.add(Flatten())

# Fully Connected (Dense) Layers:
# adds a fully connected layer with 128 units and ReLU activation.
# Dropout(0.2) randomly drops 20% of neurons.

model.add(Dense(units = 128 , activation = 'relu'))
model.add(Dropout(0.2)) 

# adds a final output layer with a single neuron using a 
# sigmoid activation for binary classification.
model.add(Dense(units = 1 , activation = 'sigmoid'))

# Compiling the Model:
# The model is compiled using the RMSprop optimizer, binary_crossentropy loss function
# (used for binary classification), and accuracy as the metric to track performance.
model.compile(optimizer = "rmsprop" , loss = 'binary_crossentropy' , metrics = ['accuracy'])

# Summary:
# prints a summary of the model architecture, showing each layer’s parameters and output shapes.
model.summary()

The model watches the validation accuracy (monitor='val_accuracy').
If accuracy doesn’t improve for 2 epochs (patience=2), the learning rate is reduced by 30% (factor=0.3).
The model won't lower the learning rate below 0.000001 (min_lr=0.000001).
This makes training slower but helps the model learn better in the long run.

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.3, min_lr=0.000001)

The model trains on batches of 32 augmented images at a time.
The training runs for 12 epochs, processing the entire dataset 12 times.
After each epoch, the model is evaluated on the validation data to check its progress.
If the validation accuracy doesn’t improve for 2 epochs, the learning rate is reduced by 30% to help the model fine-tune itself.
This process helps the model gradually improve its ability to detect pneumonia from chest X-ray images while preventing overfitting and ensuring stable learning.

In [ ]:
history = model.fit(datagen.flow(x_train,y_train, batch_size = 32) ,epochs = 12 , validation_data = datagen.flow(x_val, y_val) ,callbacks = [learning_rate_reduction])

In [ ]:
print("Loss of the model is - " , model.evaluate(x_test,y_test)[0])
print("Accuracy of the model is - " , model.evaluate(x_test,y_test)[1]*100 , "%")

# Analysis after Model Training

In [ ]:
epochs = [i for i in range(12)]
fig , ax = plt.subplots(1,2)
train_acc = history.history['accuracy']
train_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']
fig.set_size_inches(20,10)

ax[0].plot(epochs , train_acc , 'go-' , label = 'Training Accuracy')
ax[0].plot(epochs , val_acc , 'ro-' , label = 'Validation Accuracy')
ax[0].set_title('Training & Validation Accuracy')
ax[0].legend()
ax[0].set_xlabel("Epochs")
ax[0].set_ylabel("Accuracy")

ax[1].plot(epochs , train_loss , 'g-o' , label = 'Training Loss')
ax[1].plot(epochs , val_loss , 'r-o' , label = 'Validation Loss')
ax[1].set_title('Testing Accuracy & Loss')
ax[1].legend()
ax[1].set_xlabel("Epochs")
ax[1].set_ylabel("Training & Validation Loss")
plt.show()

In [ ]:
predictions =(model.predict(x_test) > 0.5).astype("int32") #binary classification
predictions = predictions.reshape(1,-1)[0]
predictions[:15]

In [ ]:
print(classification_report(y_test, predictions, target_names = ['Pneumonia (Class 0)','Normal (Class 1)']))

In [ ]:
cm = confusion_matrix(y_test,predictions)
cm

In [ ]:
cm = pd.DataFrame(cm , index = ['0','1'] , columns = ['0','1'])

In [ ]:
plt.figure(figsize = (10,10))
sns.heatmap(cm,cmap= "Blues", linecolor = 'black' , linewidth = 1 , annot = True, fmt='',xticklabels = labels,yticklabels = labels)

In [ ]:
correct = np.nonzero(predictions == y_test)[0]
incorrect = np.nonzero(predictions != y_test)[0]

**Some of the Correctly Predicted Classes**

In [ ]:
i = 0
for c in correct[:6]:
    plt.subplot(3,2,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x_test[c].reshape(150,150), cmap="gray", interpolation='none')
    plt.title("Predicted Class {},Actual Class {}".format(predictions[c], y_test[c]))
    plt.tight_layout()
    i += 1

**Some of the Incorrectly Predicted Classes**

In [ ]:
i = 0
for c in incorrect[:6]:
    plt.subplot(3,2,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(x_test[c].reshape(150,150), cmap="gray", interpolation='none')
    plt.title("Predicted Class {},Actual Class {}".format(predictions[c], y_test[c]))
    plt.tight_layout()
    i += 1